Load dataset:

In [6]:
from HumanGenomeDataset.load_dataset import load_dataset

data_df = load_dataset('dna_protein_coding_sequences')
data_df.head(2)

,ID,sequence,category
0,NM_001368254.1,GCTGAGCTGAGCTGGGGCGCAGCCGCCTGTCTGCACCGGCAGCACC...,NM
1,NM_001350977.1,TCCACAACTGAAACATCCACTTCTGAACACCATGTCCTACTACAGC...,NM


Start simulation:

In [7]:
from src.simulation import ProteinSinthesisProcess
SIM_TIME = 30

In [8]:
protein_synthesis_process = ProteinSinthesisProcess(dna_sequences_df=data_df, verbose=True)
protein_synthesis_process.run(simulation_time=SIM_TIME) # run the simulation

Eucaryotes cell initialized
Simulation environment initialized 	
Simulation started: 	
Protein synthesized
Protein synthesized
Protein synthesized
Protein synthesized


Results:

In [9]:
results_df = protein_synthesis_process.dna_sequences_df

In [10]:
results_df[results_df['peptides_cardinality']>0][
    ['polypeptides_chain_synthetized', 'polypeptides_chain_extended','peptides_cardinality']]

,polypeptides_chain_synthetized,polypeptides_chain_extended,peptides_cardinality
10090,NH2-FFGSSCPMKHLSQFISSMTPFKYPSSHTTECDLTLSTFWV-COOH,H2N-Phe-Phe-Gly-Ser-Ser-Cys-Pro-Met-Lys-His-Le...,40.0
53428,NH2-SL-COOH,H2N-Ser-Leu-COOH,2.0
58673,NH2-IIEKHTFTLEIYLLWRL-COOH,H2N-Ile-Ile-Glu-Lys-His-Thr-Phe-Thr-Leu-Glu-Il...,17.0
76594,NH2-VARAVVGPVTARVLA-COOH,H2N-Val-Ala-Arg-Ala-Val-Val-Gly-Pro-Val-Thr-Al...,15.0
